In [24]:
import pandas as pd
import numpy as np

import sandy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
endf6_file = "dec-027_Co_060.endf"

In [3]:
tape = sandy.core.endf6._FormattedFile.from_file(endf6_file)

In [4]:
tape.to_series()

MAT  MF  MT 
558  1   451    2.706000+4 5.941896+1         -1          0   ...
     8   457    2.706000+4 5.941896+1          0          0   ...
559  1   451    2.706000+4 5.941896+1         -1          0   ...
     8   457    2.706000+4 5.941896+1          1          1   ...
Name: dec-027_Co_060.endf, dtype: object

The object representation uses method `.to_series` under the hood.

tape.data

In [5]:
tape.keys

{'MAT': (558, 558, 559, 559), 'MF': (1, 8, 1, 8), 'MT': (451, 457, 451, 457)}

In [6]:
tape.mt
tape.mat
tape.mf

[451, 457]

[558, 559]

[1, 8]

In [7]:
with open(endf6_file) as f:
    text = f.read()
tape1 = sandy.core.endf6._FormattedFile.from_text(text)

In [8]:
assert tape1.data == tape.data

In [9]:
tape.filter_by(listmat=[558])
tape.filter_by(listmf=[8], listmt=[457])
tape.filter_by(listmf=[1, 8])

MAT  MF  MT 
558  1   451    2.706000+4 5.941896+1         -1          0   ...
     8   457    2.706000+4 5.941896+1          0          0   ...
Name: dec-027_Co_060.endf, dtype: object

MAT  MF  MT 
558  8   457    2.706000+4 5.941896+1          0          0   ...
559  8   457    2.706000+4 5.941896+1          1          1   ...
Name: dec-027_Co_060.endf, dtype: object

MAT  MF  MT 
558  1   451    2.706000+4 5.941896+1         -1          0   ...
     8   457    2.706000+4 5.941896+1          0          0   ...
559  1   451    2.706000+4 5.941896+1         -1          0   ...
     8   457    2.706000+4 5.941896+1          1          1   ...
Name: dec-027_Co_060.endf, dtype: object

In [10]:
new_section = "This is a new section to be added to an ENDF-6 file. Instead of text there should be fancy numbers."
tape.add_section(mat=558, mf=4, mt=999, text=new_section)

MAT  MF  MT 
558  1   451    2.706000+4 5.941896+1         -1          0   ...
     4   999    This is a new section to be added to an ENDF-6...
     8   457    2.706000+4 5.941896+1          0          0   ...
559  1   451    2.706000+4 5.941896+1         -1          0   ...
     8   457    2.706000+4 5.941896+1          1          1   ...
Name: dec-027_Co_060.endf, dtype: object

In [19]:
import io
names = ["C1", "C2", "L1", "L2", "N1", "N2"]
df = pd.read_fwf(io.StringIO(tape.data[559,8,457]), widths=[11]*6, names=names, dtype=str, na_filter=False)

In [36]:
#x = df.C1.astype(np.uint).values
x = np.array(df.C1.to_list())
# x=np.array(['some','text','this','is'])
x
x.view(np.uint8).reshape(-1, x.itemsize)

array(['2.706000+4', '6.280200+2', '5.643024+4', '2.000000+0',
       '1.000000+0', '3.000000+0', '0.000000+0', '1.000000+0',
       '5.860300+4', '3.000000+0', '4.727276+1', '8.262800+5',
       '1.000000+0', '1.332501+6', '1.000000+0', '2.158770+6',
       '1.000000+0', '0.000000+0', '1.000000+0', '7.234329+5',
       '1.000000+0', '1.549726+6', '1.000000+0', '0.000000+0',
       '1.000000+0', '4.414265+1', '3.000000+0', '7.521527+1',
       '3.000000+0', '7.005592+2', '3.000000+0', '7.466251+2',
       '3.000000+0', '6.033168+3', '3.000000+0', '6.776235+3',
       '3.000000+0', '7.523540+3', '3.000000+0', '5.089410+4',
       '3.000000+0', '5.767740+4', '3.000000+0', '5.860300+4',
       '3.000000+0', '0.000000+0', '1.000000+0', '6.841400+2',
       '3.000000+0', '7.785529+2', '3.000000+0', '7.997300+2',
       '3.000000+0', '8.110287+2', '3.000000+0', '6.873160+3',
       '3.000000+0', '6.888410+3', '3.000000+0', '7.606530+3',
       '3.000000+0', '7.608440+3', '3.000000+0', '7.666

array([[50,  0,  0, ...,  0,  0,  0],
       [54,  0,  0, ...,  0,  0,  0],
       [53,  0,  0, ...,  0,  0,  0],
       ...,
       [51,  0,  0, ...,  0,  0,  0],
       [55,  0,  0, ...,  0,  0,  0],
       [51,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [67]:
from numba import njit

@njit
def divide(a):
    res = np.empty(a.shape)
    for i in range(len(a)):
        for j in range(len(a[i])):
            if a[i,j]==43:
                res[i,j] = 1
            else:
                res[i,j] = a[i,j]
    return res

%timeit divide(x.view(np.uint8).reshape(-1, x.itemsize))  # 717 µs
A=divide(x.view(np.uint8).reshape(-1, x.itemsize))

The slowest run took 5.09 times longer than the fastest. This could mean that an intermediate result is being cached.
10.7 µs ± 8.97 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
list(map(chr, np.array(A.flatten(), np.uint8)))

['2',
 '\x00',
 '\x00',
 '\x00',
 '.',
 '\x00',
 '\x00',
 '\x00',
 '7',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '6',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '\x01',
 '\x00',
 '\x00',
 '\x00',
 '4',
 '\x00',
 '\x00',
 '\x00',
 '6',
 '\x00',
 '\x00',
 '\x00',
 '.',
 '\x00',
 '\x00',
 '\x00',
 '2',
 '\x00',
 '\x00',
 '\x00',
 '8',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '2',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '\x01',
 '\x00',
 '\x00',
 '\x00',
 '2',
 '\x00',
 '\x00',
 '\x00',
 '5',
 '\x00',
 '\x00',
 '\x00',
 '.',
 '\x00',
 '\x00',
 '\x00',
 '6',
 '\x00',
 '\x00',
 '\x00',
 '4',
 '\x00',
 '\x00',
 '\x00',
 '3',
 '\x00',
 '\x00',
 '\x00',
 '0',
 '\x00',
 '\x00',
 '\x00',
 '2',
 '\x00',
 '\x00',
 '\x00',
 '4',
 '\x00',
 '\x00',
 '\x00',
 '\x01',
 '\x00',
 '\x00',
 '\x00',
 '4',
 '\x00',
 '\x00',
 '\x00',
 

In [82]:
from numba import njit

x = np.array(df.C1.to_list())

@njit
def divide(a):
    res = np.empty(a.shape)
    for i in range(len(a)):
        res[i] = a[i]
#         a[i].contains("+")
#         if a[i].contains("+"):
#             idx = a[i].find("+")
#             res[i,idx] = "1"
    return res

#%timeit divide(x)  # 717 µs
x
A=divide(x)
A

array(['2.706000+4', '6.280200+2', '5.643024+4', '2.000000+0',
       '1.000000+0', '3.000000+0', '0.000000+0', '1.000000+0',
       '5.860300+4', '3.000000+0', '4.727276+1', '8.262800+5',
       '1.000000+0', '1.332501+6', '1.000000+0', '2.158770+6',
       '1.000000+0', '0.000000+0', '1.000000+0', '7.234329+5',
       '1.000000+0', '1.549726+6', '1.000000+0', '0.000000+0',
       '1.000000+0', '4.414265+1', '3.000000+0', '7.521527+1',
       '3.000000+0', '7.005592+2', '3.000000+0', '7.466251+2',
       '3.000000+0', '6.033168+3', '3.000000+0', '6.776235+3',
       '3.000000+0', '7.523540+3', '3.000000+0', '5.089410+4',
       '3.000000+0', '5.767740+4', '3.000000+0', '5.860300+4',
       '3.000000+0', '0.000000+0', '1.000000+0', '6.841400+2',
       '3.000000+0', '7.785529+2', '3.000000+0', '7.997300+2',
       '3.000000+0', '8.110287+2', '3.000000+0', '6.873160+3',
       '3.000000+0', '6.888410+3', '3.000000+0', '7.606530+3',
       '3.000000+0', '7.608440+3', '3.000000+0', '7.666

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mInvalid use of Function(<built-in function setitem>) with argument(s) of type(s): (array(float64, 1d, C), int64, [unichr x 10])
 * parameterized
[1mIn definition 0:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 1:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 2:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 3:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 4:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 5:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 6:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 7:[0m
[1m    All templates rejected without literals.[0m
[1mThis error is usually caused by passing an argument of a type that is unsupported by the named function.[0m[0m
[0m[1m[1] During: typing of setitem at <ipyt

In [73]:
A="1235+51"
A.find("+")

4